In [1]:
import re
import traceback
import pandas as pd
from langchain_groq import ChatGroq
from langchain.chat_models.azure_openai import AzureChatOpenAI
llm_config = {
    "deployment_name":"gpt35turbo16k",
    "model_name":"gpt-35-turbo-16k",
    "openai_api_key": "f56a89e24e7e4ce1b1369f0d70204734",
    "api_version":"2023-06-01-preview",
    "api_base" : "https://garlic-openai-api.openai.azure.com/",
    "temperature": 0.1
}
#for long context and text generation
llm_mistral = ChatGroq(temperature=0.1, 
               groq_api_key="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",
               model_name="mixtral-8x7b-32768",
               max_tokens=30000)

#for long context and text generation
llm_mistral_cc = ChatGroq(temperature=0.7, 
               groq_api_key="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",
               model_name="mixtral-8x7b-32768",
               max_tokens=30000)

#to the point answer generation
llm = AzureChatOpenAI(
    openai_api_key=llm_config["openai_api_key"],
    openai_api_base=llm_config["api_base"],
    deployment_name=llm_config["deployment_name"],
    model_name=llm_config["model_name"],
    openai_api_version=llm_config["api_version"],
    temperature=llm_config["temperature"],
)

C:\Users\lokeshkumar06\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
C:\Users\lokeshkumar06\AppData\Roaming\Python\Python312\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://garlic-openai-api.openai.azure.com/ to https://garlic-openai-api.openai.azure.com/openai.
  warnings.warn(
C:\Users\lokeshkumar06\AppData\Roaming\Python\Python312\site-packages

In [53]:
import pandas as pd
import numpy as np

class Agent:
    def __init__(self, name):
        self.name = name

    def analyze_problem(self):
        problem_statement = input("Enter the problem statement here  ")
        print('\n\n')
        return problem_statement
    
    def train_data_location(self):
        train_location = input("Enter the location of the training data CSV file: ")
        if train_location is None:
            print('No Test Data provided')
        else:
            try:
                data = pd.read_csv(train_location)
                print("Train Data loaded successfully.")
                print('\n\n')
                return data, train_location
            except Exception as e:
                print("Error loading data:", e)
                return None

    def test_data_location(self):
        test_location = input("Enter the location of the testing data CSV file: ")
        if test_location is None:
            print('No Test Data provided')
        else:
            try:
                test_data = pd.read_csv(test_location)
                print("Test Data loaded successfully.")
                print('\n\n')
                return test_data, test_location
            except Exception as e:
                print("Error loading data:", e)
                return None
    
    def identify_target_feature(self, data ,problem_statement):
        features = data.columns
        result_list = features.tolist()
        content = llm.invoke(f"Based on the problem statement and features provided,only provide back the target feature name. DO NOT provide anything else--> {problem_statement} and features {result_list}")
        target = content.content
        if target is None:
            target = input("Enter the target feature")
            print('\n\n')
        return target


    def propose_approach(self,data,problem_statement):
        X = data.drop(columns=[target_feature])
        y = data[target_feature]
        approach = llm_mistral_cc.invoke(f'Create an exhaustive list of steps to train a model that can compete with best machine learning engineers. \
        The data looks like --> {data.head()}, training features are these --> {X} and target features is this -->{y}. The problem statement is--> {problem_statement}.\
        1. Perform Exploratory analysis, like univariate, bivariate and multi-variate analysis and create graphs to show.\
        2. Perform Hypothesis testing to select best features from the dataset.\
        3. Create new features and check the corelation and colinearity.\
        4. Train a model.\
        5. DO SHAP ANALYSIS ')
        #print(approach)
        return approach

    def ensure_operation(self, problem_statement, target_feature, approach):
        print("Ensuring operation based on:")
        print('\n\n')
        #print("Problem Statement:", problem_statement)
        print('\n\n')
        print("Target Feature:", target_feature)
        print('\n\n')
        #print("Approach:", approach.content)
        print('\n\n')

In [54]:
# Instantiate an Agent object
agent = Agent(name="MyMLAgent")

# Analyze the problem statement
problem_statement = agent.analyze_problem()

# Load data from CSV file
train_data, train_location = agent.train_data_location()
test_data, test_location = agent.test_data_location()

# Identify the target feature
target_feature = agent.identify_target_feature(train_data, problem_statement)

# Propose an approach
approach = agent.propose_approach(train_data, problem_statement)

# Ensure operation
agent.ensure_operation(problem_statement, target_feature, approach)

Enter the problem statement here   In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).


Enter the location of the training data CSV file:  C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\Data\kaggle_data\titanic_train_data.csv


Train Data loaded successfully.





Enter the location of the testing data CSV file:  C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\Data\kaggle_data\titanic_test_data.csv


Test Data loaded successfully.



Ensuring operation based on:






Target Feature: Survived








## Example Details 

The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).


C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\Data\kaggle_data\titanic_train_data.csv

C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\Data\kaggle_data\titanic_test_data.csv



**************************************************************************************************************************
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 


C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\Data\kaggle_data\train_housing_prices_data.csv
C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\Data\kaggle_data\test_housing_prices_data.csv

In [47]:
print(approach.content)

Here is an exhaustive list of steps to train a model that can compete with the best machine learning engineers:

1. **Data Preprocessing:**
   - Check for missing values and handle them by imputation, deletion or using a model that can handle missing values.
   - Check for outliers and handle them by removing or transforming the data.
   - Check for data types and convert them into appropriate formats (e.g. categorical to numerical).
   - Check for skewness and normalize the data if necessary.

2. **Exploratory Data Analysis (EDA):**
   - Perform univariate analysis to understand the distribution of each feature.
   - Perform bivariate analysis to understand the relationship between two features.
   - Perform multivariate analysis to understand the relationship between multiple features.
   - Create visualizations like histograms, boxplots, scatterplots, heatmaps, etc. to support your analysis.

3. **Feature Engineering:**
   - Create new features that can help improve the model's perf

#Initialize a dictionary to store the steps
finalized_approach = {}

#Split the text by section headings
sections = re.split(r'\n\d+\. \*\*', approach.content)

#Remove the first empty string in sections list
sections.pop(0)

#Extract steps for each section
for section in sections:
    # Split each section into section name and steps
    section_parts = section.split('\n')
    section_name = section_parts[0]
    steps = [step.strip() for step in section_parts[1:] if step.strip()]
    # Store section name and steps in the dictionary
    finalized_approach[section_name] = steps

#Print the dictionary
print(finalized_approach)

In [48]:
#train_data_location = r"C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\Data\kaggle_data\titanic_train_data.csv"
#train_location
#test_location
#test_data_location = r"C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\Data\kaggle_data\titanic_test_data.csv"

In [49]:
# Loop over the dictionary and generate Python code snippets
save_model=r"C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\saved_model"    
# Generate code with LLM here
first_draft = llm.invoke(f'Only write  python code for the approach, DONOT write anything else -->  {approach.content}.  Train data is like--> {train_data.head()}, the target_variable is--> {target_feature}. Train data location is{train_location} and test data location is {test_location}. Save the model to this location--> {save_model}.')
print(first_draft.content)
#fd_code_approach = first_draft.content

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

# Step 1: Data Preprocessing
train_data = pd.read_csv("C:/Users/lokeshkumar06/OneDrive - Nagarro/Desktop/Lokesh/Data/kaggle_data/titanic_train_data.csv")
test_data = pd.read_csv("C:/Users/lokeshkumar06/OneDrive - Nagarro/Desktop/Lokesh/Data/kaggle_data/titanic_test_data.csv")

# Handle missing values
imputer = SimpleImputer(strategy='mean')
train_data['Age'] = imputer.fit_transform(train_data[['Age']])
test_data['Age'] = imputer.transform(test_data[['Age']])

# Handle categorical variables
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})

# Normalize the data
scaler = StandardScaler()
train_data[['Age', 'Fare']] = s

In [51]:
import traceback

def execute_code(code, count):
    try:
        exec(code)
        print('Code ran successfully!')
        return None, None, count
    except ValueError as ve:
        # Collect the ValueError message and stack trace
        error_message = str(ve)
        stack_trace = traceback.format_exc()
        print(error_message)
        print(stack_trace)
        return error_message, stack_trace, count + 1
    except Exception as e:
        # For all other exceptions, collect the error message and stack trace
        error_message = str(e)
        stack_trace = traceback.format_exc()
        print(error_message)
        print(stack_trace)
        return error_message, stack_trace, count + 1

# Initialize error message, stack trace, and count
error_message = "Error"
stack_trace = "Stack trace"
count = 0

# Loop until the error is sorted or a maximum number of iterations is reached
max_iterations = 3  # Set a maximum number of iterations to avoid infinite loops
while (error_message or stack_trace) and count < max_iterations:
    # Execute code and get error message, stack trace, and updated count
    error_message, stack_trace, count = execute_code(first_draft.content, count)
    
    # If there's an error, get a new draft from LLM Mistral and update the code
    if error_message or stack_trace:
        second_draft = llm_mistral_cc.invoke(f"There is an error in the code please re-write it carefully. Use reasoning and act on it. Original code -->{first_draft.content}, error-->{error_message} and and stack trace--> {stack_trace}")
        first_draft.content = second_draft.content

Accuracy: 0.8100558659217877
Code ran successfully!


In [52]:
#Define the filename to save the content
filename = r"C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\AE_program\train.py"

#Write the content to the Python file
with open(filename, "w") as file:
    file.write(first_draft.content)

print(f"Content saved to {filename}")

Content saved to C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\AE_program\train.py


## Code explanation block

In [64]:
# Define the filename
train_code = r"C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\AE_program\train.py"

# Open the file in read mode
with open(train_code, "r") as file:
    # Read the content of the file
    train_content = file.read()
code_explanation = llm_mistral_cc.invoke(f'Look into the code-->{train_content} and explain the code along with the reason in detail, explain it line by line. Take one block explain it then take another and explain it and so on. Explain it like you are explaining it to a business man who doesnt have coding experience. Explain why we did something like..why we handle missiing data and things like that.')
# Define the file path
file_path = r"C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\code_explanation\code_explanation.txt"
# Write the text to the file
with open(file_path, "w") as file:
    file.write(code_explanation.content)
print("Text saved to:", file_path)
#train_content

Text saved to: C:\Users\lokeshkumar06\OneDrive - Nagarro\Desktop\Lokesh\code_explanation\code_explanation.txt
